In [152]:
%cd ..
from llm_lora import QwenQACInference, load_model_qwen
from speedy import *
from sklearn.metrics import classification_report, f1_score


In [2]:
model, tok = load_model_qwen(
    '/anhvth5/checkpoint-llm-finetuned/qwen_14B/full_010124/', device_map=0, bf16=True)
qac = QwenQACInference(model=model, tokenizer=tok)
print('QwenQACInference loaded')

Syncing /anhvth5/checkpoint-llm-finetuned/qwen_14B/full_010124 to /home/ubuntu/.cache/huggingface/local_hub/qwen_01fe265c0f718c13
sending incremental file list

sent 1,102 bytes  received 17 bytes  2,238.00 bytes/sec
total size is 28,337,880,096  speedup is 25,324,289.63


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

QwenQACInference loaded


In [3]:
# check dtype model param, get the first param
print('Model param dtype:', next(model.parameters()).dtype)

Model param dtype: torch.bfloat16


In [39]:
ls ../../data/chat-formated-dataset/

20k_question_answer_giaothong.json
500k_qac_qae_alpha_law_mt.json
9k_QAC_LAW.json
all_chat_data_v1.pkl
all_chat_data_v2.pkl
all_chat_data_v2_1_fused.pkl
giaothong_ask_gemini_suports_or_neutral.json
giaothong_qa_long_short.json
law_query_parsing.json
nli_to_chat_augmentation.json
qa_giaothong_20k_doc_to_list_keywords_as_conversation.json
qac_to_chat_augmentation.json
tvpl_giaothong_rag_conversation_from_qwen72b.json
tvpl_giaothong_rag_nli_train.json
tvpl_giaothong_rag_nli_val.json
tvpl_giaothong_rag_nli_vietnamese_train.json
tvpl_giaothong_rag_nli_vietnamese_val.json


In [52]:
nli_data = load_by_ext(
    '../../data/chat-formated-dataset/tvpl_giaothong_rag_nli_vietnamese_train.json')

In [63]:

# Example:  {'conversations': [{'from': 'user', 'value': 'Please determine the relationship between the following premise and hypothesis:\nPremise: ```“Đi bão” là hành động rất đông người dân đổ xô ra đường phố cùng nhau ăn mừng chiến thắng, thường xuất hiện trong những sự kiện bóng đá có sự tham gia của các Đội tuyển bóng đá nam Việt Nam. Hoạt động này không có định nghĩa, khái niệm trong các văn bản quy phạm pháp luật. Tuy nhiên, nếu đi bão gây混乱 giao thông, gây tổn hại đến tài sản công hoặc của cá nhân, người tham gia có thể bị xử lý theo quy định của pháp luật.```\n\nHypothesis: ```Văn bản hợp nhất 15/VBHN-VPQH 2019 Luật Giao thông đường bộ\nChương I |  Điều 3. Giải thích từ ngữ\nTrong Luật này, các từ ngữ dưới đây được hiểu như sau:\n10. Dải phân cách là bộ phận của đường để phân chia mặt đường thành hai chiều xe chạy riêng biệt hoặc để phân chia phần đường của xe cơ giới và xe thô sơ. Dải phân cách gồm loại cố định và loại di động.\nTrong Luật này, các từ ngữ dưới đây được hiểu như sau:\n11. Nơi đường giao nhau cùng mức (sau đây gọi là nơi đường giao nhau) là nơi hai hay nhiều đường bộ gặp nhau trên cùng một mặt phẳng, gồm cả mặt bằng hình thành vị trí giao nhau đó.\nTrong Luật này, các từ ngữ dưới đây được hiểu như sau:\n12. Đường cao tốc là đường dành cho xe cơ giới, có dải phân cách chia đường cho xe chạy hai chiều riêng biệt; không giao nhau cùng mức với một hoặc các đường khác; được bố trí đầy đủ trang thiết bị phục vụ, bảo đảm giao thông liên tục, an toàn, rút ngắn thời gian hành trình và chỉ cho xe ra, vào ở những điểm nhất định.\nTrong Luật này, các từ ngữ dưới đây được hiểu như sau:\n13. Đường chính là đường bảo đảm giao thông chủ yếu trong khu vực.\nTrong Luật này, các từ ngữ dưới đây được hiểu như sau:\n14. Đường nhánh là đường nối vào đường chính.```'}, {'from': 'assistant', 'value': '4. Neutral/Irrelevant'}]}
def pretty_print_nli_data(nli_data):
    for conv in nli_data['conversations']:
        print(conv['from'], '>>>', conv['value'])


pretty_print_nli_data(nli_data[3])

user >>> 
Câu hỏi (question): ```Trước khi xe ô tô của cơ quan Bảo hiểm xã hội Việt Nam lưu hành có phải kiểm tra tình trạng kỹ thuật, lượng xăng dầu của xe không?```

Câu trả lời mong đợi (expected answer): ```Trước khi xe ô tô của cơ quan Bảo hiểm xã hội Việt Nam lưu hành, lái xe phải kiểm tra tình trạng kỹ thuật và lượng xăng dầu của xe. Nếu xe không đủ điều kiện vận hành, lái xe phải báo cáo và đề xuất phương án xử lý với Đội trưởng Đội xe.```

Tài liệu đã truy xuất (retrived document): ```Thông tư 12/2020/TT-BGTVT quản lý hoạt động vận tải bằng xe ô tô và dịch vụ hỗ trợ vận tải đường bộ  ...
Chương II |  Điều 4. Quy định chi tiết về quy trình bảo đảm an toàn giao thông
Quy trình đảm bảo an toàn giao thông phải đảm bảo theo trình tự các bước và nội dung tối thiểu như sau:
3. Sau khi được giao nhiệm vụ và trước khi cho xe khởi hành, người lái xe được giao nhiệm vụ vận chuyển phải thực hiện kiểm tra đảm bảo tình trạng an toàn kỹ thuật của phương tiện (riêng đơn vị kinh doanh vận tải 

In [64]:
# print(nli_data[3]['system'])

In [5]:
NLI_TEMPLATE = '''Please determine the relationship between the following premise and hypothesis:
Premise: ```Question: {question}
Answer: {answer}
```
Hypothesis: ```{document}```'''

In [6]:
from guidance import *

In [248]:
llm = models.QwenChat(model, tok)

2024-01-06 11:36:34.337 | INFO     | guidance.models.transformers._qwen:__init__:47 - Set tokenizer.eos_token_id = tokenizer.eod_id


In [8]:
qac_test = load_by_ext('/anhvth5/data/qac-data/za_qac/qac_test.jsonl')
qac_test[0]

{'id': 'test_1001p1',
 'question': 'Con trai đầu của Ngô Quyền tên gì',
 'title': 'Nhà Ngô',
 'text': 'Ngô Quyền hạ thành Đại La , giết Công Tiễn rồi bày trận trên sông Bạch Đằng đón quân Nam Hán .',
 'label': False}

In [9]:
item = qac_test[15]
question = item['question']
document = '{} | {}'.format(item['title'], item['text'])
out = qac(question, document, '5level',
          expected_answer='10 vị vua', force=True, decode=True)


def qac_one_item(item):
    question = item['question']
    document = '{} | {}'.format(item['title'], item['text'])
    out = qac(question, document, '5level',
              expected_answer='', force=True, decode=True)
    return out

## Forward batches

In [161]:

documents = ['{} | {}'.format(item['title'], item['text'])
            for item in qac_test]
questions = [item['question'] for item in qac_test]
outputs = qac.forward(questions, documents,
                    template='binary', return_scores=True)

100%|██████████| 110/110 [00:40<00:00,  2.72it/s]


In [162]:
# outputs

In [163]:
labels = [item['label'] for item in qac_test]

In [164]:
best_thr = 0
for thr in np.arange(0.2, .8, 0.05):
    # print('Threshold:', thr)
    preds = [1 if score > thr else 0 for score in outputs]
    score = f1_score(labels, preds)
    best_thr = max(best_thr, score)
    print(f1_score(labels, preds))
    # print('-----------------------------------------')
best_thr

0.8821410190427175
0.8930817610062893
0.8993074054342036
0.9024258760107817
0.9028757460661964
0.9055982436882547
0.9020475926950747
0.9041248606465998
0.9022046353872244
0.9016486640136441
0.8963938179736692
0.8920023350846469
0.8835820895522387


0.9055982436882547

# Use NLI as QAC

In [126]:
def convert_cls_to_score(cls_out):
    score = 0
    for k, v in cls_out.items():
        score += (int(k)-1)/4 * v
    return score


scores = [convert_cls_to_score(cls_out) for cls_out in outputs]
best_thr = 0
for thr in np.arange(0.2, 1, 0.02):
    print('Threshold:', thr)
    preds = [1 if score > thr else 0 for score in scores]
    score = f1_score(labels, preds)
    best_thr = max(best_thr, score)
    print(classification_report(labels, preds))
    print('-----------------------------------------')

Threshold: 0.2
              precision    recall  f1-score   support

       False       1.00      0.15      0.27      2614
        True       0.29      1.00      0.45       898

    accuracy                           0.37      3512
   macro avg       0.64      0.58      0.36      3512
weighted avg       0.82      0.37      0.31      3512

-----------------------------------------
Threshold: 0.22
              precision    recall  f1-score   support

       False       1.00      0.19      0.31      2614
        True       0.30      1.00      0.46       898

    accuracy                           0.39      3512
   macro avg       0.65      0.59      0.39      3512
weighted avg       0.82      0.39      0.35      3512

-----------------------------------------
Threshold: 0.24
              precision    recall  f1-score   support

       False       1.00      0.27      0.43      2614
        True       0.32      1.00      0.49       898

    accuracy                           0.46      35

In [165]:
squad_data = load_by_ext(
    '/anhvth5/data/chat-data/za_chat/envi_squad_train.json'
)

In [246]:
from guidance import *
squad_data[0]
# {'conversations': [{'from': 'user',
#    'value': '{"document": "Phạm Văn Đồng | Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4 năm 2000) là Thủ tướng đầu tiên của nước Cộng hòa Xã hội chủ nghĩa Việt Nam từ năm 1976 (từ năm 1981 gọi là Chủ tịch Hội đồng Bộ trưởng) cho đến khi nghỉ hưu năm 1987. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa từ năm 1955 đến năm 1976. Ông là vị Thủ tướng Việt Nam tại vị lâu nhất (1955–1987). Ông là học trò, cộng sự của Chủ tịch Hồ Chí Minh. Ông có tên gọi thân mật là Tô, đây từng là bí danh của ông. Ông còn có tên gọi là Lâm Bá Kiệt khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm (Chủ nhiệm là Hồ Học Lãm).", "question": "<blank>", "answer": "Lâm Bá Kiệt", "language": "vi"}'},
#   {'from': 'assistant',
#    'value': 'question: Tên gọi nào được Phạm Văn Đồng sử dụng khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm?'}],
#  'system': 'Task: Field in the blank'}

def pretty_print(squad_data):
    with system():
        lm = llm + squad_data.get('system', 'You are a helpful assistant.')
    with user():
        lm += squad_data['conversations'][0]['value']
    with assistant():
        lm += squad_data['conversations'][1]['value']
    return lm
# lm = pretty_print_squad_data(squad_data[0])

In [194]:
data_user = '''{"document": "Phạm Văn Đồng | Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4 năm 2000) là Thủ tướng đầu tiên của nước Cộng hòa Xã hội chủ nghĩa Việt Nam từ năm 1976 (từ năm 1981 gọi là Chủ tịch Hội đồng Bộ trưởng) cho đến khi nghỉ hưu năm 1987. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa từ năm 1955 đến năm 1976. Ông là vị Thủ tướng Việt Nam tại vị lâu nhất (1955–1987). Ông là học trò, cộng sự của Chủ tịch Hồ Chí Minh. Ông có tên gọi thân mật là Tô, đây từng là bí danh của ông. Ông còn có tên gọi là Lâm Bá Kiệt khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm (Chủ nhiệm là Hồ Học Lãm).", "question": "Tên gọi khác của Phạm Văn đồng", "answer": "<blank>", "language": "vi"}'''
out = model.chat(tok,data_user, history=[]);
out[0]

'answer: Tô'

In [195]:
rag = load_by_ext('/anhvth5/data/chat-formated-dataset/tvpl_giaothong_rag_conversation_from_qwen72b.json')

In [204]:
output = model.chat(tok, rag[1]['conversations'][0]['value'], history=[])

In [208]:
print(rag[1]['conversations'][0]['value'])

Query: 01 Biển số định danh có được cấp cho nhiều xe?
Retrieved documents: {
  "138635": "Thông tư 24/2023/TT-BCA cấp thu hồi đăng ký biển số xe cơ giới mới nhất\nChương I |  Điều 3. Nguyên tắc đăng ký xe\n1. Xe cơ giới có nguồn gốc hợp pháp, bảo đảm tiêu chuẩn chất lượng an toàn kỹ thuật và bảo vệ môi trường thì được đăng ký, cấp một bộ biển số xe theo quy định tại Thông tư này.\n2. Chủ xe là tổ chức, cá nhân có trụ sở, nơi cư trú (nơi đăng ký thường trú, tạm trú) tại địa phương nào thì đăng ký xe tại cơ quan đăng ký xe thuộc địa phương đó; trừ trường hợp quy định tại khoản 14 Điều 3 Thông tư này .\n3. Biển số xe được cấp và quản lý theo mã định danh của chủ xe (sau đây gọi là biển số định danh). Biển số định danh là biển số có ký hiệu, seri biển số, kích thước của chữ và số, màu biển số theo quy định tại Thông tư này.\n4. Đối với chủ xe là công dân Việt Nam thì biển số xe được quản lý theo số định danh cá nhân.\n5. Đối với chủ xe là người nước ngoài thì biển số xe được quản lý theo s

In [211]:

questions_to_docs = {}
for item in qac_test:
    question = item['question']
    document = '{} | {}'.format(item['title'], item['text'])
    if question not in questions_to_docs:
        questions_to_docs[question] = []
    questions_to_docs[question].append(document)
    
for query, retrieved_docs in questions_to_docs.items():
    template = '''Query: {query}
Retrieved documents: {docs}
Guide:
    - Hãy phân tích sau đó đưa ra kết luận về câu hỏi này
    - Response language: Vietnamese'''
    retrieved_docs = dict(enumerate(retrieved_docs, 1))
    user = template.format(query=query, docs=json.dumps(retrieved_docs, ensure_ascii=False, indent=4))
    break

In [222]:
squad = load_by_ext('/anhvth5/data/chat-data/za_chat/envi_squad_train.json')

In [225]:
nli = load_by_ext('../../data/chat-formated-dataset/nli_to_chat_augmentation.json')

In [245]:
model.chat(tok, nli[24]['conversations'][0]['value'], history=[])[0]

'{\n    "answer": true\n}'

In [230]:
data = load_by_ext('../../data/chat-formated-dataset/nli_to_chat_augmentation.json')

In [251]:
# pretty_print(data[27])

In [253]:
# x = '''    "question": "Trình độ đào tạo của Chuyên viên cao cấp về quản lý nhân viên hàng không phải đáp ứng các yêu cầu nào?",
#     "text": "Quyết định 777/QĐ-CHK 2023 phê duyệt tạm thời bản mô tả công việc Cục Hàng không Việt Nam |  - Thực hiện các nhiệm vụ được giao khác.\n- Có bằng tốt nghiệp đại học trở lên một trong các ngành, chuyên ngành: Hệ thống điện, điện tử viễn thông, công nghệ thông tin, tự động hoá, điều khiển tự động, kỹ thuật hàng không, cơ khí, chế tạo máy, xây dựng công trình, quản lý công nghiệp, môi trường, các chuyên ngành về công nghệ\n- Có bằng tốt nghiệp cao cấp lý luận chính trị hoặc bằng cao cấp lý luận chính trị - hành chính hoặc có giấy xác nhận trình độ lý luận chính trị tương đương cao cấp lý luận chính trị của cơ quan có thẩm quyền.\n- Đảm bảo các yêu cầu, điều kiện của ngạch chuyên viên cao cấp theo quy định.\n- Có kỹ năng sử dụng công nghệ thông tin cơ bản.\n- Có kỹ năng sử dụng ngoại ngữ ở trình độ bậc 3 khung năng lực ngoại ngữ Việt Nam.\n- Nắm vững chủ trương, đường lối của Đảng, chính sách pháp luật của Nhà nước về chuyên ngành thuộc chức năng, nhiệm vụ của cơ quan.\n- Có chuyên môn, nghiệp vụ cao nhất trong ngành, lĩnh vực công tác; có hiểu biết về vị trí, vai trò và tác động của nghiệp vụ chuyên ngành.\n- Có năng lực dự đoán xu hướng và xu thế phát triển ngành, lĩnh vực chuyên môn; có kinh nghiệm tham mưu xử lý các vấn đề về chuyên môn, nghiệp vụ.",'''
# model.chat(tok, f'Please answer the question base on the provided content: {x}', history=[])[0]